## Paper : Interpretation of Electrocardiogram Heartbeat by CNN and GRU

In [210]:
import wfdb
import os
import pandas as pd
import wfdb.processing as wp
import numpy as np
import pickle
from biosppy.signals import ecg, tools


import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from torch import nn, optim

import pytorch_model_summary

from sklearn.preprocessing import MinMaxScaler as mms

import matplotlib.pyplot as plt
import matplotlib

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIbLE_DEVICES"] = "0"

In [211]:
# Interpretation of Electrocardiogram Heartbeat by CNN and GRU (not Use)
class LFEM(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size):
        super(LFEM, self).__init__()
        
        self.lfem_seq = nn.Sequential(
            nn.Conv1d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size, stride=1, padding=1),
            nn.Conv1d(in_channels=out_channel, out_channels=out_channel, kernel_size=kernel_size, stride=1, padding=1),
            nn.BatchNorm1d(out_channel),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
    
    def forward(self, x):
        return self.lfem_seq(x)

class LFEM_Stack(nn.Module):
    def __init__(self, num_layer=6):
        super(LFEM_Stack, self).__init__()
        
        self.num_layer = num_layer
        self.lfem_list = []
        layer_size=[64,64,128,128,256,256]
        for i in range(num_layer-1):
            self.lfem_list.append(LFEM(in_channel=layer_size[i], out_channel=layer_size[i+1], kernel_size=3))
        self.lfem_list.append(LFEM(in_channel=layer_size[5], out_channel=layer_size[5], kernel_size=3))
            
    def forward(self, x):
        for i in range(self.num_layer):
            x = self.lfem_list[i](x)
            
        return x

In [214]:
class conv_gru(nn.Module):
    def __init__(self, in_channel, out_channel, batch, hidden_size):
        super(conv_gru, self).__init__()
        
        self.input_seq = nn.Sequential(
            nn.Conv1d(in_channels=in_channel,out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )
        
        
        
        self.LFEM1 = LFEM(32,64,3)
        self.LFEM2 = LFEM(64,64,3)
        self.LFEM3 = LFEM(64,128,3)
        self.LFEM4 = LFEM(128,128,3)
        self.LFEM5 = LFEM(128,256,3)
        self.LFEM6 = LFEM(256,256,3)
        
        
        self.dropout = nn.Dropout(0.3)
        
        self.gru = nn.GRU(input_size=1280 ,batch_first=True, hidden_size=hidden_size, num_layers=2, bidirectional=False)
        self.dense_seq = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256, out_channel)
        )
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.input_seq(x)
        x = self.LFEM1(x)
        x = self.LFEM2(x)
        x = self.LFEM3(x)
        x = self.LFEM4(x)
        x = self.LFEM5(x)
        x = self.LFEM6(x)
        x = self.dropout(x)
        x = x.reshape(x.size(0),-1)
        x = x.unsqueeze(1)
        x, _ = self.gru(x)
        x = x.squeeze(1)
        x = self.dense_seq(x)
        return self.softmax(x)

In [215]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = conv_gru(in_channel=1, out_channel=6, batch=32, hidden_size=256)
model.to(device)

conv_gru(
  (input_seq): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (LFEM1): LFEM(
    (lfem_seq): Sequential(
      (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (LFEM2): LFEM(
    (lfem_seq): Sequential(
      (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  

In [216]:
print(pytorch_model_summary.summary(model,torch.randn(32,1,320).cuda(), show_input=True, batch_size=32, show_hierarchical=True))

-----------------------------------------------------------------------
      Layer (type)         Input Shape         Param #     Tr. Param #
          Conv1d-1        [32, 1, 320]             128             128
     BatchNorm1d-2       [32, 32, 320]              64              64
            ReLU-3       [32, 32, 320]               0               0
            LFEM-4       [32, 32, 320]          18,688          18,688
            LFEM-5       [32, 64, 160]          24,832          24,832
            LFEM-6        [32, 64, 80]          74,240          74,240
            LFEM-7       [32, 128, 40]          98,816          98,816
            LFEM-8       [32, 128, 20]         295,936         295,936
            LFEM-9       [32, 256, 10]         394,240         394,240
        Dropout-10        [32, 256, 5]               0               0
            GRU-11       [32, 1, 1280]       1,575,936       1,575,936
         Linear-12           [32, 256]          65,792          65,792
     

In [217]:
# Pickle Data Load

def load_data():
    pkl_path = "./pickle/"

    data_x, data_y = [] ,[] 
    for pkl_file in os.listdir(pkl_path):
        if os.path.isdir(pkl_path+pkl_file):
            continue
        with open(pkl_path+pkl_file, "rb") as f:
            data = pickle.load(f)
            for i,d in enumerate(data["Beats"]):
                data_x.append(d)
                data_y.append(data["symbol"][i])
#             print(torch.tensor(data["Beats"]).shape)
#             print(torch.tensor(data["symbol"]).shape)
    return torch.tensor(data_x), torch.tensor(data_y)

x_data, y_data = load_data()
print(x_data.shape, y_data.shape)

train_data = torch.utils.data.TensorDataset(x_data,y_data)
train_len = x_data.shape[0]
# print(train_len)
val_len = int(train_len * 0.2)
# print(val_len)
train_len -= val_len
train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_len, val_len])
print(train_len, val_len)

lr = 1e-5
epochs = 30
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = conv_gru(in_channel=1,out_channel=6,batch=256,hidden_size=256)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

best_acc, best_epoch = 0, 0
global_step = 0

torch.Size([112599, 320]) torch.Size([112599])
90080 22519


In [218]:
x_data.shape

torch.Size([112599, 320])

In [219]:
y_data.shape

torch.Size([112599])

In [220]:
train_data = torch.utils.data.TensorDataset(x_data,y_data)
train_len = x_data.shape[0]
print(train_len)
val_len = int(train_len * 0.2)
print(val_len)
train_len -= val_len
print(train_len)
train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_len, val_len])

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

112599
22519
90080


In [221]:
print(len(train_dataset))
print(len(val_dataset))

90080
22519


In [222]:
def evaluate_acc(model, val_loader):
    
    model.eval()
    correct = 0
    total = len(val_loader.dataset)
    val_bar = tqdm(val_loader)
    val_loss = 0
    for step, (x, y) in enumerate(val_bar):
        x = x.unsqueeze(dim=1).to(device).float()
        y = y.to(device).long()
        
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
#             print(pred.shape, y.shape)
            loss = criterion(logits, y)
            val_loss += loss.item()
        
        correct += torch.eq(pred, y).sum().float().item()
    
    return correct/total, val_loss

In [223]:
from tqdm import tqdm
from pytorchtools import EarlyStopping

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

early_stopping = EarlyStopping(patience = 3, verbose = True)


loss_list = []

for ep in range(epochs):
    train_bar = tqdm(train_loader)
    for step, (x,y) in enumerate(train_bar):
        
        x = x.unsqueeze(dim=1)
#         print("input,",x.shape, y.shape)
        x, y = x.to(device).float(), y.to(device).long()
        model.train()
        logits = model(x)
        loss = criterion(logits, y)
        loss_list.append(loss)
#         print(loss)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        train_bar.desc = "Train Epoch[{}/{}] loss: {:.3f}".format(ep+1, epochs, loss)
        global_step +=1
    
#     model.eval()
#     val_loss = 0
    
#     val_bar = tqdm(val_loader)
#     for step, (x_, y_) in enumerate(val_bar):
#         x_ = x_.unsqueeze(dim=1).to(device).float()
#         y_pred = model(x_)
#         y_ = y_.to(device).long()
#         print(y_.shape, y_pred.shape)
#         loss = criterion(y_pred, y_)
        
#         val_loss += loss.item()
    
    acc, val_loss = evaluate_acc(model, val_loader)
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "./best.mdl")
    
        
    print("validation ACC :",acc)
    early_stopping(val_loss, model)
    
    if early_stopping.early_stop:
        break

Train Epoch[2/30] loss: 1.267:   1%|█▋                                                                                                                                                  | 4/352 [00:00<00:09, 37.10it/s]

validation ACC : 0.8697988365380346
Validation loss decreased (inf --> 107.744096).  Saving model ...


Train Epoch[3/30] loss: 1.200:   1%|█▋                                                                                                                                                  | 4/352 [00:00<00:09, 37.13it/s]

validation ACC : 0.9184688485279098
Validation loss decreased (107.744096 --> 101.822823).  Saving model ...


Train Epoch[4/30] loss: 1.149:   1%|█▋                                                                                                                                                  | 4/352 [00:00<00:09, 36.81it/s]

...

validation ACC : 0.976153470402771
EarlyStopping counter: 1 out of 3


Train Epoch[26/30] loss: 1.090:   1%|█▋                                                                                                                                                 | 4/352 [00:00<00:09, 37.90it/s]

validation ACC : 0.9766863537457259
EarlyStopping counter: 2 out of 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:00<00:00, 121.75it/s]

validation ACC : 0.9766863537457259
EarlyStopping counter: 3 out of 3


In [224]:
def load_data():
    pkl_path = "./pickle/test/"
    data_x, data_y = [] ,[] 
    for pkl_file in os.listdir(pkl_path):
        if os.path.isdir(pkl_path+pkl_file):
            continue
        with open(pkl_path+pkl_file, "rb") as f:
            data = pickle.load(f)
            for i,d in enumerate(data["Beats"]):
                data_x.append(d)
                data_y.append(data["symbol"][i])
#             print(torch.tensor(data["Beats"]).shape)
#             print(torch.tensor(data["symbol"]).shape)
    return torch.tensor(data_x), torch.tensor(data_y)

x_test_data, y_test_data = load_data()
print(x_data.shape, y_data.shape)

test_dataset = torch.utils.data.TensorDataset(x_test_data, y_test_data)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

torch.Size([112599, 320]) torch.Size([112599])


In [225]:
test_acc, _ = evaluate_acc(model, test_loader)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:00<00:00, 122.58it/s]


In [226]:
test_acc

0.8926109565848349

## Accuracy를 향상시키는 방법
1. Denoising
2. Augmentation
 * Scailing
 * Resampling
 * Clipping

Augmentation 방법이 좀 어려움